# IMBD Data Analytics with Python
## Guia de Análise Exploratória de Dados com Python e Linguagem SQL

In [3]:
# Fazendo os imports dos pacotes necessários para análise
import re
import time
import sqlite3
import pycountry
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")
sns.set_theme(style = "whitegrid")

In [6]:
%%time 
# magic function has to be the first thing in the IPython (Jupyter) cell

# Baixando e carregando Dados do IMDB com a lib imdb-sqlite
!imdb-sqlite

2021-07-13 16:09:02,707 GET https://datasets.imdbws.com/name.basics.tsv.gz -> downloads/name.basics.tsv.gz
2021-07-13 16:09:39,559 GET https://datasets.imdbws.com/title.basics.tsv.gz -> downloads/title.basics.tsv.gz
2021-07-13 16:09:59,631 GET https://datasets.imdbws.com/title.akas.tsv.gz -> downloads/title.akas.tsv.gz
2021-07-13 16:10:34,557 GET https://datasets.imdbws.com/title.principals.tsv.gz -> downloads/title.principals.tsv.gz
2021-07-13 16:11:31,567 GET https://datasets.imdbws.com/title.episode.tsv.gz -> downloads/title.episode.tsv.gz
2021-07-13 16:11:36,381 GET https://datasets.imdbws.com/title.ratings.tsv.gz -> downloads/title.ratings.tsv.gz
2021-07-13 16:11:37,231 Populating database: imdb.db
2021-07-13 16:11:37,237 Applying schema
2021-07-13 16:11:37,240 Importing file: downloads/name.basics.tsv.gz
2021-07-13 16:11:37,241 Reading number of rows ...
2021-07-13 16:11:43,326 Inserting rows into table: people
100%|█████████████████████████| 11070282/11070282 [02:24<00:00, 76625